<a href="https://colab.research.google.com/github/ethandavenport/Optimization-I-Project-2/blob/main/linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 51.8 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [5]:
url = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-2/main/roi_company1.csv"
df = pd.read_csv(url)